# Exercise 3.2

In [2]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
N_elements=5
K=np.zeros((N_elements,N_elements))
F=np.zeros(N_elements)
# A=[0,1,2]
x_A=np.linspace(0,N_elements-1,N_elements)
S=1-x_A

LM = np.zeros((2, N_elements), dtype=np.int64)
for e in range(N_elements):
    for a in [1,2]:
        for b in [1,2]:
            k_e=(-1)**(a+b)/(x_A[e+1]-x_A[e])
            f_e=(x_A[e+1]-x_A[e])/6*np.array()
    if e==0:
        # Treat first element differently due to BC
        LM[0, e] = -1 # Left hand node of first element
        # is not considered thanks to BC.
        LM[1, e] = 0 # the first equation
    else:
        # Left node of this element is
        # right node of previous element
        LM[0, e] = LM[1, e-1]
        LM[1, e] = LM[0, e] + 1

BV={'alpha':1,
'beta':2}





# S=1-x_A

In [24]:
print(x_A)

[0. 1. 2. 3. 4.]


In [25]:
Ne = 10
nodes = np.array([0, *sorted(np.random.rand(Ne-1)), 1])
LM = np.zeros((2, Ne), dtype=np.int64)
for e in range(Ne):
    if e==0:
        LM[0, e] = -1
        LM[1, e] = 0
    else:
        LM[0, e] = LM[1, e-1]
        LM[1, e] = LM[0, e] + 1
K = np.zeros((Ne, Ne))
F = np.zeros((Ne,))
for e in range(Ne):
    k_e = stiffness(nodes[e:e+2])
    f_e = force(nodes[e:e+2], S)
    for a in range(2):
        A = LM[a, e]
        for b in range(2):
            B = LM[b, e]
            if (A >= 0) and (B >= 0):
                K[A, B] += k_e[a, b]
        if (A >= 0):
            F[A] += f_e[a]
    # Modify force vector for Dirichlet BC
    if e == 0:
        F[0] -= alpha * k_e[1, 0]
# Modify force vector for Neumann BC
F[-1] += beta
# Solve
Psi_A = np.zeros_like(nodes)
Psi_A[0] = alpha
Psi_A[1:] = np.linalg.solve(K, F)

NameError: name 'stiffness' is not defined